In [20]:
## Data prep

import os
import random

train_root = "CUB_formatted/train"
val_root = "CUB_formatted/val"
roots = [train_root, val_root]
num_classes = 200

if num_classes != 200:
    all_classes = sorted(os.listdir(train_root))
    selected_classes = all_classes[:num_classes]
    print(selected_classes)
#selected_classes = random.sample(all_classes, num_classes)

In [21]:
## Torchvision data formatting

from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset
import torch

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

filtered_datasets = []

for root in roots:
    full_dataset = datasets.ImageFolder(root, transform=transform)

    if num_classes == 200:
        filtered_datasets.append(full_dataset)
    else:
        selected_classes_idx = [full_dataset.class_to_idx[c] for c in selected_classes]
        label_remap = {orig: new for new, orig in enumerate(sorted(selected_classes_idx))}

        filtered_data = [
            (img, label_remap[label])
            for img, label in full_dataset
            if label in selected_classes_idx
        ]

        filtered_datasets.append(filtered_data) 

train_data, val_data = filtered_datasets
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = DataLoader(val_data, batch_size=32, shuffle=True)

In [22]:
## Torchvision model construction

import torchvision.models as models
from torchvision.models import ResNet18_Weights
import torch.nn as nn
import torch.optim as optim

model = models.resnet18(weights=ResNet18_Weights.DEFAULT)

model.fc = nn.Linear(model.fc.in_features, num_classes)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [23]:
## Torchvision training

num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}")

KeyboardInterrupt: 

In [13]:
## Torchvision validation

model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Validation Accuracy: {100 * correct / total:.2f}%")

Validation Accuracy: 54.95%


In [ ]:
## Tensorflow

In [ ]:
## Uniform Dummy